# Prompting on Liar Dataset

## Dataset Setup

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import datasets

liar = datasets.load_dataset("liar")
liar

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})

In [4]:
train = liar["train"]
test = liar["test"]
val = liar["validation"]

In [5]:
full_liar = datasets.concatenate_datasets([train, test, val])
full_liar

/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 12836
})

## Model Loading

In [6]:
falcon = "tiiuae/falcon-7b-instruct"
llama = "meta-llama/Llama-2-7b-chat-hf"
mistral = "mistralai/Mistral-7B-Instruct-v0.2"
orca = "microsoft/Orca-2-7b"

In [7]:
# change this depending on experiment
model_name = llama

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=config, device_map="auto"
)

/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Experiment

### Experiment Utils

In [9]:
from transformers import PreTrainedTokenizer
from typing import Dict

# split into True/False
LABEL_MAP = {
    0:"A",
    1:"B"
}

# 0 : False
# 1 : Half True
# 2 : Mostly True
# 3 : True
# 4 : Barely True
# 5 : Pants on Fire
def to_binary_label(entry):
    if entry['label'] in [3, 2, 1]:
        entry['label'] = 0
    else:
        entry['label'] = 1
    return entry

def was_correct(
    decoded:str, entry: Dict[str, int]
) -> bool:
    return LABEL_MAP[entry["label"]] in decoded

In [10]:
full_liar = full_liar.map(to_binary_label)

In [11]:
n_examples = 5

In [12]:
import random
random.seed(1770)
entries = random.choices(list(range(len(full_liar))), k=n_examples)

def to_zero_shot_prompt(entry: Dict[str, str]) -> str:
    speaker = entry["speaker"].replace("-", " ").title()
    statement = entry["statement"].lstrip("Says ")

    prompt = f"""Please select the option that most closely describes the following claim by {speaker}:\n{statement}\n\nA) True\nB) False\n\nChoice: ("""
    return prompt

def to_n_shot_prompt(n: int, entry: Dict[str, str]) -> str:
    examples = ""
    for i in range(n):
        examples += to_zero_shot_prompt(full_liar[entries[i]]) + LABEL_MAP[full_liar[entries[i]]['label']] + "\n\n"
    prompt = to_zero_shot_prompt(entry)
    return examples + prompt


In [15]:
responses: Dict[str, list] = {}
def workflow(idx, entry: dict, model, k:int=0, verbose: bool = False) -> bool:
    prompt = to_n_shot_prompt(k, entry)

    # encode input, move it to cuda, then generate
    encoded_input = tokenizer(prompt, return_tensors="pt")
    encoded_input = {item: val.cuda() for item, val in encoded_input.items()}
    generation = model.generate(
        **encoded_input,
        max_new_tokens=1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    # log the prompt and response if verbose
    if verbose:
        print(tokenizer.batch_decode(generation)[0])

    decoded = tokenizer.decode(generation[0, -1])
    correct = was_correct(decoded, entry)

    if decoded not in responses:
        responses[decoded] = []
    responses[decoded].append(idx)

    if verbose:
        print(
            "The model was",
            "correct" if correct else "incorrect",
            " - responded",
            tokenizer.decode(generation[0, -1]),
            "and answer should have been",
            LABEL_MAP[entry["label"]],
        )
    return correct

### Verify it works

In [16]:
import random
workflow(0, train[random.randint(0, len(train) - 1)], model, verbose=True, k=n_examples)

/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s> Please select the option that most closely describes the following claim by Kendrick Meek:
Warren Buffett called credit default swaps financial weapons of mass destruction. And Greene was the first individual to use them.

A) True
B) False

Choice: (A

Please select the option that most closely describes the following claim by Bill Nelson:
Many state and federal agencies have such navigators involved in helping folks maneuver through the often complex processes associated with filing benefits claims, for example -- even buying health insurance.

A) True
B) False

Choice: (A

Please select the option that most closely describes the following claim by George Lemieux:
(George) LeMieux never requested a single earmark and pushed to ban them all.

A) True
B) False

Choice: (A

Please select the option that most closely describes the following claim by Bill Oreilly:
We researched to find out if anybody on Fox News had ever said youre going to jail if you dont buy health insurance. Nobody

True

### Run Experiment

Results of zero-shot prompting the models

In [20]:
from tqdm import tqdm

num_correct = 0
responses = {}
for idx, entry in enumerate((prog := tqdm(full_liar))):
    if idx in entries:
        continue  # don't include items that were in the examples
    
    correct = workflow(idx, entry, model, k=n_examples)
    if correct:
        num_correct += 1
    prog.set_postfix_str(f"acc: {num_correct/(idx+1):.3f}")

  0%|                                                 | 0/12836 [00:00<?, ?it/s]/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/016854656/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|█████████████████████████| 12836/12836 [53:29<00:00,  4.00it/s, acc: 0.505]


In [ ]:
# log results
with open(f"{n_examples}_shot.txt", "a") as file:
    file.write(f"{model_name} : {num_correct}/{len(full_liar)-len(entries)}\n")

In [ ]:
# print results up till now
with open(f"{n_examples}_shot.txt", "r") as file:
    print(file.read())

In [21]:
import pickle
pickle.dump(responses, open(f"{model_name[model_name.index('/')+1:]}_responses_bool.pk", "wb"))